In [1]:
def f1():
    x = 10
    
    def f2():
        nonlocal x
        x = 20
        print('x in f2 is', x)
    
    f2()
    print('x in f1 is', x)

x = 5
f1()
print('Global x:', x)

x in f2 is 20
x in f1 is 20
Global x: 5


In [7]:
def make_withdraw(balance):
    def withdraw(amount):
        nonlocal balance
        if amount > balance:
            return 'Insufficient funds'
        balance = balance - amount
        return balance
    return withdraw

wd = make_withdraw(20)
wd(5)

15

In [2]:
def f1():
    x = 10
    
    def f2():
        x = 20
        print('x in f2 is', x)
    
    f2()
    print('x in f1 is', x)

x = 5
f1()
print('Global x:', x)

x in f2 is 20
x in f1 is 10
Global x: 5


In [5]:
def f1():
    x = 10
    
    def f2():
        
        print('x in f2 is', x)
    
    f2()
    print('x in f1 is', x)

x = 5
f1()
print('Global x:', x)

x in f2 is 10
x in f1 is 10
Global x: 5


In [1]:
def dictionary():
    
    records = []
    def getitem(key):
        matches = [r for r in records if r[0] == key]
        if len(matches) == 1:
            key, value = matches[0]
            return value
    def setitem(key, value):
        nonlocal records
        non_matches = [r for r in records if r[0] != key]
        records = non_matches + [[key, value]]
    def dispatch(message, key=None, value=None):
        if message == 'getitem':
            return getitem(key)
        elif message == 'setitem':
            setitem(key, value)
    return dispatch

In [5]:
my_dict = dictionary()

In [6]:
my_dict('setitem', key='LIU', value=23)
my_dict('setitem', key='WANG', value=21)

In [7]:
my_dict('getitem', key='LIU')

23

In [9]:
def account(initial_balance):
    def deposit(amount):
        dispatch['balance'] += amount
        return dispatch['balance']
    def withdraw(amount):
        if amount > dispatch['balance']:
            return 'Insufficient funds'
        dispatch['balance'] -= amount
        return dispatch['balance']
    dispatch = {'deposit': deposit,
                'withdraw': withdraw,
                'balance': initial_balance
                }
    return dispatch

In [10]:
def withdraw(account, amount):
    return account['withdraw'](amount)
def deposit(account, amount):
    return account['deposit'](amount)
def check_balance(account):
    return account['balance']

In [11]:
a = account(20)
deposit(a, 5)
withdraw(a, 17)
check_balance(a)

8

In [ ]:
def adder(a, b, c)

In [ ]:
def convert(c, f):
    '''connect c to f with constraints to convert from Celsius to Fahrenheit'''
    u, v, w, x, y = [connector() forr _ in range 5]
    

In [ ]:
celsius = connector('Celsius')
fahrenheit = connector('Fahrenheit')